## Observations and Insights

## Dependencies and starter code

In [ ]:
# %matplotlib notebook

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

# Study data files
mouse_metadata = "data/Mouse_metadata.csv"
study_results = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata)
study_results = pd.read_csv(study_results)

# Combine the data into a single dataset
metadata_results_combined_df = pd.merge(mouse_metadata, study_results, on='Mouse ID', how='outer')

# Show the DataFrame
metadata_results_combined_df

## Summary statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Group by the Drug Regimen to get some stats.
regimen_group = metadata_results_combined_df.groupby(['Drug Regimen'])

# Get the average, create a dataframe and rename the column
volume_summary = regimen_group['Tumor Volume (mm3)'].mean()
volume_summary_df = pd.DataFrame(volume_summary)
volume_summary_df = volume_summary_df.rename(columns={'Tumor Volume (mm3)':'Mean Tumor Volume (mm3)'})

# Get the other stats and show the dataframe
volume_summary_df['Median Tumor Volume (mm3)'] = regimen_group['Tumor Volume (mm3)'].median()
volume_summary_df['Variance Tumor Volume (mm3)'] = regimen_group['Tumor Volume (mm3)'].var()
volume_summary_df['Stand. Dev. Tumor Volume (mm3)'] = regimen_group['Tumor Volume (mm3)'].std()
volume_summary_df['SEM Tumor Volume (mm3)'] = regimen_group['Tumor Volume (mm3)'].sem()
volume_summary_df

## Bar plots

In [ ]:
# Generate a bar plot showing number of data points for each treatment regimen using pyplot
# Get the number of data points from the previous groupby and get the count
number_of_data_points = regimen_group['Tumor Volume (mm3)'].count()

# Get a list of the index from the volume stats dataframe and use that list to create the xaxis for the bar plot
drug_regimen = volume_summary_df.index.values.tolist()
x_axis = np.arange(len(drug_regimen))

# Create a bar plot with the data using matplotlib.plot and prettyfy the plot
plt.bar(x_axis, number_of_data_points, tick_label=drug_regimen, width=.5)
plt.xticks(rotation='vertical')
plt.title('Number of Data Points per Drug Regimen')
plt.ylabel('Number of Data Points')
plt.xlabel('Drug Regimen')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Generate a bar plot showing number of data points for each treatment regimen using pandas
# Create a dataframe of the groupby where I got the count of values form the above cell
data_point_count_df = pd.DataFrame(number_of_data_points)

# Create a bar plot using pandas plot of the data and prettyfy it
Count_PDPlot = data_point_count_df.plot(kind='bar', grid=True, legend=False)
plt.title('Number of Data Points per Drug Regimen')
Count_PDPlot.set_ylabel('Number of Data Points')
plt.tight_layout()
plt.show()

## Pie plots

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
# Create a dataframe with only one value for each mouse, I did this by droping the duplicates in the Mouse ID column
# In this case, I only kept the last row for latter use.
unique_mouse = metadata_results_combined_df.drop_duplicates(subset=['Mouse ID'], keep='last')

# I created a groupby of the sex column and used the .count() method to get the number of males and females, and put this into a dataframe
sex_ratio_group = unique_mouse.groupby(['Sex']).count()
sex_ratio_df = pd.DataFrame(sex_ratio_group['Mouse ID'])

# I created a list of the count of males and females and the word "male", "female" correlated to that count value
sex_ratio = list(sex_ratio_df['Mouse ID'])
labels = sex_ratio_df.index.values.tolist()

# I then ploted this in a pie chart using matplotlib
Ratio_MPLPlot = plt.pie(sex_ratio, labels=labels, autopct='%1.1f%%')
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
# Taking the sex_ratio_df from above, I got a list of the keys which are the labels "male", 'female'
sex_ratio_df
gender_list = sex_ratio_df.keys()

# I then ploted this in a pie chart using pandas from the dataframe directly
Ratio_PDPlot = sex_ratio_df.plot(kind='pie', y=gender_list, legend=False, autopct='%1.1f%%')
a=Ratio_PDPlot.set_ylabel('')
plt.show()

## Quartiles, outliers and boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. 
# Capomulin, Ramicane, Infubinol, and Ceftamin.
# using the merged dataframe, pull out rows that have the 4 regimens that we are interested in
metadata_results_combined_df
prom_regimens = metadata_results_combined_df[metadata_results_combined_df['Drug Regimen']
                                             .isin(['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin'])] 

# to get the final value, groupby the Mouse ID and just take the last value
# I can do this because I sorted this df by the Timepoint
prom_regimens.sort_values(by="Timepoint")
final_volume = prom_regimens.groupby(['Mouse ID']).tail(1)

# sort the values by the tumor volume for easier access to the max and min volume later on
final_volume = final_volume.sort_values(by=['Tumor Volume (mm3)'])

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# loop through each drug regimen and calculate the IQR
for drug in list(set(final_volume['Drug Regimen'])):
    quartiles = final_volume.loc[final_volume['Drug Regimen']==drug]['Tumor Volume (mm3)'].quantile([0.25, 0.5, 0.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq - lowerq

    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)

#     Grab the max and min by using .head(1) and .tail(1) since the tumor volume is sorted
    max_tumor_volume = float(final_volume.loc[final_volume['Drug Regimen']==drug]['Tumor Volume (mm3)'].tail(1))
    min_tumor_volume = float(final_volume.loc[final_volume['Drug Regimen']==drug]['Tumor Volume (mm3)'].head(1))
       
#     determine if the max and min is outside of the upper and lower bounds, and give print accordingly
    if (max_tumor_volume > upper_bound) | (min_tumor_volume < lower_bound):
        print(f"{drug} has at least one potential outlier.")
    else:
        print(f"{drug} does not have any outliers.")


In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

# Using Matplotlib, generate a box and whisker plot of the final tumor volume for all four treatment regimens and highlight any potential outliers in the plot by changing their color and style.
# Hint: All four box plots should be within the same figure. Use this Matplotlib documentation page for help with changing the style of the outliers.

# create a list of the tumor volumes of all of the drug regimens that we are interested in
bp_data = [final_volume['Tumor Volume (mm3)'].loc[final_volume['Drug Regimen']=='Ramicane'],
           final_volume['Tumor Volume (mm3)'].loc[final_volume['Drug Regimen']=='Capomulin'],
           final_volume['Tumor Volume (mm3)'].loc[final_volume['Drug Regimen']=='Infubinol'],
           final_volume['Tumor Volume (mm3)'].loc[final_volume['Drug Regimen']=='Ceftamin']
          ]

# create veriables for plotting
fig, ax = plt.subplots()
# add plot text for understanding the plot shown
ax.set_title("Final Tumor Volume per Drug Regimen")
ax.set_ylabel("Final Tumor Volume (mm3)")
ax.set_xticklabels(["Ramicane", "Capomulin", "Infubinol", "Ceftamin"])
ax.set_xlabel("Drug Regimen")
# create a boxplot of our data created in the list above
bp = plt.boxplot(bp_data)
plt.tight_layout()
plt.grid(False)
plt.show()

## Line and scatter plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
# create a new dataframe for the mouse that I thought was interesting.
Capomulinf966_df = metadata_results_combined_df.loc[(metadata_results_combined_df['Drug Regimen']=='Capomulin')&
                            (metadata_results_combined_df['Mouse ID']=='f966')]

# create a line plot from that dataframe using pandas
capomulin_line_plot = Capomulinf966_df.plot.line(x='Timepoint', y='Tumor Volume (mm3)', legend=False)
title = plt.title(f"For Mouse with ID f966")
ylabel = plt.ylabel("Tumor Volume (mm3)")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
# Create a dataframe of the drug regimen that we are interested in
Capomulin_df = metadata_results_combined_df.loc[metadata_results_combined_df['Drug Regimen']=='Capomulin']

# calculate the average values for each mouse
AverageTumorVolme_perMouse_df = Capomulin_df.groupby(['Mouse ID']).mean()

# create a scatter plot using pandas
scatter = AverageTumorVolme_perMouse_df.plot.scatter(x='Tumor Volume (mm3)', y='Weight (g)')
title = plt.title("Mouse Weight vs Average Tumor Volume")
plt.show()

In [ ]:
# Calculate the correlation coefficient and linear regression model for mouse weight and average tumor volume for the 
# Capomulin regimen

# using the pearson method get the correlation coefficient and print it for the Capoulin drug regimen
CorrelationCoefficient = round(st.pearsonr(AverageTumorVolme_perMouse_df['Tumor Volume (mm3)'],
                                           AverageTumorVolme_perMouse_df['Weight (g)'])[0],2)
print(f"The correlation between mouse weight and average tumor volume for the Capomulin regimen is {CorrelationCoefficient}.")


In [ ]:
# create the veriables for the linear regression of the mouse weight vs tumor volume
wt_slope, wt_int, wt_r, wt_p, wt_std_err = st.linregress(AverageTumorVolme_perMouse_df['Tumor Volume (mm3)'],
                                                            AverageTumorVolme_perMouse_df['Weight (g)']
                                                           )

# Create equation of line
wt_fit = wt_slope * AverageTumorVolme_perMouse_df['Tumor Volume (mm3)'] + wt_int
line_eq = (f"y = {round(wt_slope,2)}(g/mm3)x {round(wt_int,2)}(g)")

In [ ]:
# Plot the linear model and equation on top of scatter plot 
# create the scatter plot
scatter = AverageTumorVolme_perMouse_df.plot.scatter(x='Tumor Volume (mm3)', y='Weight (g)')
title = plt.title("Mouse Weight vs Average Tumor Volume")
# create the linear regression plot
plt.plot(AverageTumorVolme_perMouse_df['Tumor Volume (mm3)'], wt_fit, color="grey")
plt.annotate(line_eq,(38,15.5), fontsize=15, color="grey")

plt.show()